In [10]:
import torch
import numpy as np

# Scrabble Functions
Functions to get Scrabble scores for all words in computer dictionary

In [125]:
def scrabble_letter(l):
    if l in "eaotinrslu":
        return 1
    elif l in "dg":
        return 2
    elif l in "cmbp":
        return 3
    elif l in "hfwyv":
        return 4
    elif l in "k":
        return 5
    elif l in "jx":
        return 8
    elif l in "qz":
        return 10
    else:
        return 0

def scrabble_words(words):
    return [sum([scrabble_letter(l) for l in word]) for word in words]

def get_words():
    words = open("/usr/share/dict/words","r").read().lower().split('\n')
    np.random.shuffle(words)
    return words

def score_pairs():
    words = get_words()
    scores = scrabble_words(words)
    max_score = max(scores)
    return list(zip(words,scores)),max_score

word_scores,max_score = score_pairs()

# One-Hotters
Convert values from above into tensors
(adapted from [this tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html))

In [126]:
n_letters = 26
all_letters = 'abcdefghijklmnopqrstuvwxyz'

def letter_to_index(letter):
    return all_letters.find(letter)

def word_to_tensor(word):
    tensor = torch.zeros(len(word), 1, n_letters)
    for li, letter in enumerate(word):
        tensor[li][0][letter_to_index(letter)] = 1
    return tensor

def tensor_words(word_scores,max_score):
    pairs = []
    for word,score in word_scores:
        pairs.append([word_to_tensor(word),word,torch.tensor(float(score)/max_score)])
    return pairs

tensor_scores = tensor_words(word_scores,max_score)

# RNN
Define the RNN (adapted from [this tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html))

In [127]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output.reshape(1,), hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# RNN Train Function
(adapted from [this tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html))

In [128]:
def train(word,value,lr):
    hidden = rnn.initHidden()
    rnn.zero_grad()

    for i in range(word.size()[0]):
        output, hidden = rnn(word[i], hidden)

    loss = criterion(output, value)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-lr, p.grad.data)

    return output, loss.item()

# Full Training Loop
(adapted from [this tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html))

In [ ]:
n_iters = 1000
print_every = 50
current_loss = 0
lr = .05

#rnn = RNN(n_letters, 512, 1)
#criterion = torch.nn.functional.mse_loss

for i,(tensor,word,value) in enumerate(tensor_scores):
    output, loss = train(tensor,value,lr)
    current_loss += loss

    # Print iter number, loss, name and guess
    if i % 50 == 0:
        print("{}: Actual: {}, Predicted: {}".format(word,int(value*max_score),int(output.data*max_score)))

cully: Actual: 10, Predicted: 9
refectorary: Actual: 19, Predicted: 19
douse: Actual: 6, Predicted: 5
antiprism: Actual: 13, Predicted: 13
propwood: Actual: 16, Predicted: 16
nympholepsy: Actual: 26, Predicted: 26
monoamide: Actual: 14, Predicted: 13
dendric: Actual: 11, Predicted: 11
twopence: Actual: 15, Predicted: 15
impostume: Actual: 15, Predicted: 15
zoonite: Actual: 16, Predicted: 15
reshake: Actual: 14, Predicted: 14
splanchnosclerosis: Actual: 27, Predicted: 26
urethritis: Actual: 13, Predicted: 13
bathymetric: Actual: 23, Predicted: 22
paralgesia: Actual: 13, Predicted: 13
phonographist: Actual: 24, Predicted: 24
puncture: Actual: 12, Predicted: 12
copperware: Actual: 19, Predicted: 18
benempted: Actual: 16, Predicted: 15
homeotypic: Actual: 22, Predicted: 21
besmearer: Actual: 13, Predicted: 13
lavage: Actual: 10, Predicted: 9
schnabelkanne: Actual: 24, Predicted: 24
rinde: Actual: 6, Predicted: 5
pantropic: Actual: 15, Predicted: 15
debunker: Actual: 15, Predicted: 14
hup: 

saltant: Actual: 7, Predicted: 7
barbican: Actual: 14, Predicted: 13
ideologue: Actual: 11, Predicted: 11
optative: Actual: 13, Predicted: 13
detractor: Actual: 12, Predicted: 12
troublemaker: Actual: 20, Predicted: 20
emulative: Actual: 14, Predicted: 14
uppull: Actual: 10, Predicted: 10
weatherboard: Actual: 21, Predicted: 21
undispensing: Actual: 16, Predicted: 16
koltunnor: Actual: 13, Predicted: 12
matralia: Actual: 10, Predicted: 9
monohydric: Actual: 21, Predicted: 20
phallism: Actual: 15, Predicted: 14
korec: Actual: 11, Predicted: 10
imbue: Actual: 9, Predicted: 9
monarchy: Actual: 18, Predicted: 18
archencephala: Actual: 25, Predicted: 24
midstreet: Actual: 12, Predicted: 11
subjectivism: Actual: 28, Predicted: 27
favored: Actual: 14, Predicted: 13
satinpod: Actual: 11, Predicted: 11
moucharaby: Actual: 22, Predicted: 22
carboxylic: Actual: 26, Predicted: 26
eutectoid: Actual: 12, Predicted: 11
fever: Actual: 11, Predicted: 10
diminishment: Actual: 20, Predicted: 20
sunsettin

uncharitably: Actual: 22, Predicted: 22
auramine: Actual: 10, Predicted: 9
uppertendom: Actual: 18, Predicted: 17
spheroidic: Actual: 18, Predicted: 18
peccant: Actual: 13, Predicted: 12
caryophyllaceae: Actual: 30, Predicted: 30
condor: Actual: 9, Predicted: 9
fantasticalness: Actual: 20, Predicted: 19
dueling: Actual: 9, Predicted: 9
zymogene: Actual: 23, Predicted: 22
myrrhic: Actual: 17, Predicted: 17
isothere: Actual: 11, Predicted: 10
subreport: Actual: 13, Predicted: 13
tranquillize: Actual: 30, Predicted: 30
blightingly: Actual: 21, Predicted: 20
annullable: Actual: 12, Predicted: 12
pygarg: Actual: 13, Predicted: 12
inartistical: Actual: 14, Predicted: 14
hyperbulia: Actual: 20, Predicted: 20
anacharis: Actual: 14, Predicted: 14
chalcostibite: Actual: 22, Predicted: 21
impercipient: Actual: 20, Predicted: 19
choirman: Actual: 15, Predicted: 15
membranelike: Actual: 22, Predicted: 22
recept: Actual: 10, Predicted: 9
mesail: Actual: 8, Predicted: 8
willinghood: Actual: 19, Predi

desirability: Actual: 18, Predicted: 18
strongbox: Actual: 19, Predicted: 19
fritt: Actual: 8, Predicted: 7
scorningly: Actual: 16, Predicted: 15
superaxillary: Actual: 25, Predicted: 24
horonite: Actual: 11, Predicted: 11
redecoration: Actual: 15, Predicted: 14
tenoroon: Actual: 8, Predicted: 8
uprest: Actual: 8, Predicted: 8
floralia: Actual: 11, Predicted: 10
bulger: Actual: 9, Predicted: 9
temporohyoid: Actual: 23, Predicted: 22
slitty: Actual: 9, Predicted: 9
ticking: Actual: 14, Predicted: 13
circumvolve: Actual: 23, Predicted: 22
randomly: Actual: 14, Predicted: 13
bandikai: Actual: 15, Predicted: 15
oncetta: Actual: 9, Predicted: 9
ventromedial: Actual: 18, Predicted: 17
snowslide: Actual: 13, Predicted: 12
dendrocoela: Actual: 15, Predicted: 15
crapple: Actual: 13, Predicted: 13
arseniate: Actual: 9, Predicted: 8
beflea: Actual: 11, Predicted: 11
horizontalism: Actual: 27, Predicted: 26
shewbread: Actual: 18, Predicted: 17
oscillatoriaceae: Actual: 20, Predicted: 19
predilecti